In [1]:
import requests

# Agromonitoring API details
API_KEY = "dbc1bd74bb612ba70f4ced84451dca15"
POLYGON_URL = "http://api.agromonitoring.com/agro/1.0/polygons"

# Define coordinates for a 20 ha area in Nebraska
# Note: Coordinates are examples and can be adjusted
polygon_coordinates = {
    "name": "Test Area 20ha",
    "geo_json": {
        "type": "Feature",
        "properties": {},
        "geometry": {
            "type": "Polygon",
            "coordinates": [[
                [-98.4842, 40.5705],  # Point 1
                [-98.4842, 40.5905],  # Point 2
                [-98.4642, 40.5905],  # Point 3
                [-98.4642, 40.5705],  # Point 4
                [-98.4842, 40.5705]   # Close the polygon
            ]]
        }
    }
}

# Create polygon and get polyid
def create_polygon():
    response = requests.post(
        POLYGON_URL,
        json=polygon_coordinates,
        params={"appid": API_KEY}
    )
    if response.status_code == 200:
        data = response.json()
        print(f"Polygon created successfully: {data}")
        return data["id"]  # Return polyid
    else:
        print(f"Failed to create polygon: {response.text}")
        return None

if __name__ == "__main__":
    polyid = create_polygon()
    print(f"Polygon ID: {id}")

Failed to create polygon: {"name":"UnprocessableEntityError","message":"Your polygon is duplicated your already existed polygon '678d07589c1d0d5e12fe3ada'. If you need to create the same polygon - pass the flag 'duplicated=true'"}
Polygon ID: <built-in function id>


In [ ]:
import requests
import json
import time
from datetime import datetime
from kafka import KafkaProducer

# Kafka producer configuration
producer = KafkaProducer(
    bootstrap_servers=["localhost:9092"],  # Đổi địa chỉ nếu cần
    value_serializer=lambda v: json.dumps(v).encode("utf-8")
)

# Agromonitoring API details
API_KEY = "dbc1bd74bb612ba70f4ced84451dca15"
BASE_URL_WEATHER = "https://api.agromonitoring.com/agro/1.0/weather"
BASE_URL_SOIL = "http://api.agromonitoring.com/agro/1.0/soil"
POLYID = "678d07589c1d0d5e12fe3ada"  # Thay bằng polyid từ bước tạo polygon

# Fetch weather data
def fetch_weather():
    weather_response = requests.get(
        f"{BASE_URL_WEATHER}?lat=40.5705&lon=-98.4842&appid={API_KEY}"
    )
    weather_data = weather_response.json()
    return {
        "temperature": weather_data.get("main", {}).get("temp"),
        "humidity_air": weather_data.get("main", {}).get("humidity"),
        "pressure": weather_data.get("main", {}).get("pressure"),
    }

# Fetch soil data
def fetch_soil():
    soil_response = requests.get(
        f"{BASE_URL_SOIL}?polyid={POLYID}&appid={API_KEY}"
    )
    soil_data = soil_response.json()
    return {"soil_moisture": soil_data.get("moisture")}

# Combine data into a single JSON
def combine_data():
    weather = fetch_weather()
    soil = fetch_soil()
    combined = {**weather, **soil, "timestamp": datetime.utcnow().isoformat()}
    return combined

# Send data to Kafka topic
def send_to_kafka(data, topic="weather_data"):
    producer.send(topic, value=data)
    print(f"Data sent to Kafka topic '{topic}': {data}")

if __name__ == "__main__":
    try:
        while True:
            combined_data = combine_data()
            send_to_kafka(combined_data)
            time.sleep(3600)
    except KeyboardInterrupt:
        print("Producer stopped.")

Data sent to Kafka topic 'weather_data': {'temperature': 251.12, 'humidity_air': 59, 'pressure': 1044, 'soil_moisture': 0.107, 'timestamp': '2025-01-21T04:45:58.370233'}
Data sent to Kafka topic 'weather_data': {'temperature': 251.12, 'humidity_air': 59, 'pressure': 1044, 'soil_moisture': 0.107, 'timestamp': '2025-01-21T05:46:01.529798'}
